### Train CNN Model 3 Classes

In [2]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras import layers

DATA_PATH = "/Users/danielporras/Musica/musica_analysis/models/data.json_3_classes"

def add_noise(data):
    noise_factor = 0.003 * np.random.randn(*data.shape)
    data_noise = data + noise_factor
    return data_noise

def load_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["features"])
    y = np.array(data["labels"])

    smote = SMOTE()
    X_resampled, y_resampled = smote.fit_resample(X, y)
    
    indices = np.random.randint(0, X.shape[0], size=int(X.shape[0] * 0.12))
    X[indices] = add_noise(X[indices])
    
    return X_resampled, y_resampled

def prepare_datasets(test_size, validation_size):
    X, y = load_data(DATA_PATH)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    X_train = X_train[..., np.newaxis, np.newaxis]
    X_validation = X_validation[..., np.newaxis, np.newaxis]
    X_test = X_test[..., np.newaxis, np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test


def build_model_v2(input_shape, dropout_rate=0.4, l2_regularizer=0.0005):
    model = tf.keras.Sequential()

    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.BatchNormalization())

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate))

    model.add(layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularizer)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate))

    model.add(layers.Dense(3, activation='softmax'))  # Adjusted to 3 classes

    return model

if __name__ == "__main__":
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

    model = build_model_v2(input_shape, dropout_rate=0.4, l2_regularizer=0.0005)

    optimiser = tf.keras.optimizers.RMSprop(learning_rate=0.0003)

    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-6)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()

    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation),
                        batch_size=32, epochs=100, callbacks=[lr_scheduler, early_stopping])

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)

    model.save('/Users/danielporras/Musica/music_x/best_model.h7')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 1, 64)         640       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 13, 1, 64)         0         
 g2D)                                                            
                                                                 
 batch_normalization_6 (Bat  (None, 13, 1, 64)         256       
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 1, 128)        73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 7, 1, 128)         0         
 g2D)                                                            
                                                      

INFO:tensorflow:Assets written to: /Users/danielporras/Musica/music_x/best_model.h7/assets
